# Integer recognition - Attempt 02

Expectation: Recognize digit 2 & 4 distinctively with a ML code entirely implemented by me from the scratch<br/>
<hr/>
Data set for "2" : D:/ENTC/SEM_4/EN2550 - Fundamentals of Image Processing and Machine Vision/~images/int-rec/2/ <br/>
Data set for "4" : D:/ENTC/SEM_4/EN2550 - Fundamentals of Image Processing and Machine Vision/~images/int-rec/4/


In [114]:
dataFolder2 = r"D:/ENTC/SEM_4/EN2550 - Fundamentals of Image Processing and Machine Vision/~images/int-rec/2/"
dataFiles2 = os.listdir(dataFolder2)

### Importing libraries

In [77]:
import cv2 as cv
import time
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
import scipy.signal as sig
import pprint
import json
import scipy.stats as scistat

### Generating random kernels/filters to detect basic shapes from data images

also saving them as JSON for back up

In [31]:
ksize =5
kernelCount = 50
cmap="gray"

filters = []
tries =0
while len(filters) < kernelCount:
    tries += 1
    testKernel = np.random.rand(ksize,ksize)
    testKernel -= np.mean(testKernel)
    testKernel = 2*np.array(testKernel > 0,np.float32) 
    testKernel -= 1
    testKernel[testKernel.shape[0]//2,testKernel.shape[1]//2] -= np.sum(testKernel)
    
    #checking if this kernel already in the filters array
    isUnique =1
    for filter in filters:
        if np.all(filter==testKernel):
            isUnique = 0
    if isUnique:
        filters.append(testKernel)

        
#saving filters as json 
filtersJsonFolder = "D:/ENTC/SEM_4/EN2550 - Fundamentals of Image Processing and Machine Vision/Image-processing-fundamentals/integer-rec/"
filtersAsArrays = [] #because numpy arrays cannot be converted to JSON obj
for filter in filters:
    filtersAsArrays.append(filter.tolist())
f=open(filtersJsonFolder + str(time.time()) + ".txt","w")
f.write(json.dumps(filtersAsArrays))
f.close()



### Feature threshold values
After filtering, the results should be cleaned. So the non-dominant activations will be zeroed using these thresholds.


In [32]:
featureTresVals = [0.5,0.6,0.7,0.8,0.9]

## Training for "2"


In [115]:
trainingDataFileFolder = dataFolder2
trainingFiles = dataFiles2
dataFileCount = len(trainingFiles)

### Parameters 
There are $N_k$ kernels and $N_t$ threshold  values, resulting in $N_kN_t$ processes.<br/>
Let result from one of those process be $R_{ij}$ ; $i = kernel\ index$ and $j = threshold\ index$ 
<br/>
Output should be a weighted sum of all process results ($R_{ij}$)
<br/>
Those weights are the parameters, which are changed to minimize the error
<br/>
As $R_{ij}$ are 2D, these parameters $\alpha_{ij}$ also are 2D
<br/>
In this "params" variable, all the 2D-parameters are saved as a matrix.So variable param is a 4D-array.


<hr/>

First the initial parameters are generated/guessed as the averageZeroed normalized average of all the results (per process).<br/>
Which is better than starting from a random 2D array!

### Initializing param

In [54]:
#initial parameters are generated/guessed as the normalized average of all the results (per process)
params = []
for filterIndex in range(len(filters)):
    thisFilter = filters[filterIndex]
    parameterArrayForThisKernel =[]
    for featureTres in featureTresVals:
        avgForProcess = 0 #will be a 2D image (average)
        for fileIndex in range(len(trainingFiles)):
            filePath = trainingDataFileFolder + trainingFiles[fileIndex]
            dataImg = cv.imread(filePath,0)
            assert dataImg is not None
            dataImg = dataImg.astype(np.float32)
            
            filtered = cv.filter2D(dataImg,-1,thisFilter)
            thresholderMask = filtered > np.max(filtered) * featureTres
            thresholdedResult = filtered*thresholderMask
            
            
            if not fileIndex:
                avgForProcess = thresholdedResult/dataFileCount
            else:
                avgForProcess += thresholdedResult/dataFileCount
                
        avgForProcess /= np.sum(avgForProcess) #normalizing
        avgForProcess -= np.mean(avgForProcess) #mean zeroing
        parameterArrayForThisKernel.append(avgForProcess)
    params.append(parameterArrayForThisKernel)    
        

In [56]:
params[0][0][50][4]

-0.0003138732

### Fine tuning "params" the 4D array
"prams" has $N_k$ X $N_t$ X $h_{data}$ X $w_{data}$  parameters
<br/>
Only for each process, each 2D_parameter will be scaled. That is; their distribution is only depending on the average of results.


In [100]:
paramRes = []
for filterIndex in range(len(filters)):
    thisFilter = filters[filterIndex]
    parameterArrayForThisKernel =[]
    for featureTresIndex in range(len(featureTresVals)):
        featureTres = featureTresVals[featureTresIndex]
        
        #the parameter (2D) for this process (consisting by this kernel and treshold)
        parameter2D = params[filterIndex][featureTresIndex] #normalized,averageZeroed
        
        for fileIndex in range(len(trainingFiles)):
            filePath = trainingDataFileFolder + trainingFiles[fileIndex]
            dataImg = cv.imread(filePath,0)
            assert dataImg is not None
            dataImg = dataImg.astype(np.float32)
            
            filtered = cv.filter2D(dataImg,-1,thisFilter)
            thresholderMask = filtered > np.max(filtered) * featureTres
            thresholdedResult = filtered*thresholderMask
            thresholdedResultNorm = thresholdedResult / np.sum(thresholdedResult)#normalizing
            thresholdedResultNormAVGZERO = thresholdedResultNorm - np.mean(thresholdedResultNorm) #averageZeroing
            correlation = np.sum(thresholdedResultNormAVGZERO*parameter2D)
            print(correlation)
            paramRes.append(correlation)
            break
                
           
        

0.0019511478
0.002365036
0.003461623
0.0050414195
0.017436909
0.0018981409
0.0022031409
0.0028245435
0.0045064725
0.008498594
0.00169882
0.0018711186
0.0021022481
0.0028269992
0.004601555
0.0022007055
0.0025869622
0.0028677052
0.0028585582
0.002854515
0.0020149448
0.0026686992
0.004380267
0.005731773
0.008971195
0.0036948726
0.0053247386
0.007259367
0.0127097145
0.0153794745
0.0021007648
0.002644267
0.0031293142
0.0043135798
0.007907291
0.0029042969
0.005194563
0.008211745
0.01707378
0.034965567
0.0017413619
0.0021236148
0.002241756
0.004081334
0.007651536
0.0020975887
0.0023499716
0.0023504077
0.002668179
0.0038502226
0.0018911053
0.00249955
0.0056682997
0.020853
0.03728337
0.002544811
0.0029122322
0.003002642
0.0034847427
0.0060499837
0.0025359835
0.0035782836
0.006838162
0.015628863
0.03168341
0.0021063692
0.003175661
0.005446742
0.016759507
0.020912929
0.0017797165
0.002493152
0.0041748355
0.0077590914
0.016215345
0.0022369334
0.0026905993
0.0033476902
0.0061570685
0.011980214
0.00

In [120]:

for fileIndex in range(len(trainingFiles)):
    paramRes = []
    filePath = trainingDataFileFolder + trainingFiles[fileIndex]
    dataImg = cv.imread(filePath,0)
    assert dataImg is not None
    dataImg = dataImg.astype(np.float32)
    
    for filterIndex in range(len(filters)):
        corrsForThisFilter =[]
        thisFilter = filters[filterIndex]
        parameterArrayForThisKernel =[]
        for featureTresIndex in range(len(featureTresVals)):
            featureTres = featureTresVals[featureTresIndex]

            #the parameter (2D) for this process (consisting by this kernel and treshold)
            parameter2D = params[filterIndex][featureTresIndex] #normalized,averageZeroed

        
            
            filtered = cv.filter2D(dataImg,-1,thisFilter)
            thresholderMask = filtered > np.max(filtered) * featureTres
            thresholdedResult = filtered*thresholderMask
            thresholdedResultNorm = thresholdedResult / np.sum(thresholdedResult)#normalizing
            thresholdedResultNormAVGZERO = thresholdedResultNorm - np.mean(thresholdedResultNorm) #averageZeroing
            correlation = np.sum(thresholdedResultNormAVGZERO*parameter2D)
            #print(correlation)
            corrsForThisFilter.append(correlation)
        paramRes.append(corrsForThisFilter)
        
    npParamRes = np.array(paramRes)
    dominant = np.logical_and(npParamRes > 0,npParamRes>0.001)
    print("c",np.sum(dominant))
                
           
        

c 250
c 250
c 250
c 245
c 249
c 248
c 249
c 250
c 247
c 218
c 209
c 215
c 243
c 242
c 249
c 244
c 249
c 233
c 204
c 249
c 250
c 195
c 229
c 164
c 249
c 250
c 231
c 225
c 238
c 249
c 232
c 250
c 228
c 86
c 139
c 250
c 240
c 249
c 250
c 250
c 250
c 208
c 223
c 250
c 248
c 220
c 249
c 249
c 250
c 130
c 191
c 233
c 248
c 220
c 246
c 125
c 250
c 136
c 245
c 250
c 247
c 250
c 195
c 250
c 243
c 248
c 250
c 249
c 249
c 248
c 122
c 250
c 233
c 242
c 175
c 248
c 250
c 239
c 250
c 240
c 242
c 250
c 232
c 249
c 250
c 250
c 250
c 149
c 246
c 250
c 245
c 242
c 250
c 231
c 250


In [101]:
print("total params",len(paramRes))
npParamRes = np.array(paramRes)

np.sum(npParamRes <0)

total params 250


0